# Sphere Manifold

Optimization on a sphere manifold, taken from: https://www.manopt.org/manifold_documentation_sphere.html


## Problem Description

Rayleigh quotient problem:
$$\min_{x}x^TAx,$$
$$\text{s.t. }x^Tx=1,$$
where $A\in R^{n\times n}$ is a symmetric matrix. 

## Modules Importing
Import all necessary modules and add NCVX src folder to system path.

In [1]:
import time
import torch
import sys
## Adding NCVX directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/NCVX')
from ncvx import ncvx
from ncvxStruct import Options, GeneralStruct 

## Data Generation 
Specify torch device, and generate data

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [2]:
device = torch.device( 'cuda')
torch.manual_seed(0)
n = 500
A = torch.randn((n,n)).to(device=device, dtype=torch.double)
A = .5*(A+A.T)

## Problem Definition

Specify optimization variables, and objective and constraint(s).

Note: please strictly follow the format of comb_fn, which will be used in the NCVX main algortihm.

In [3]:
# variables and corresponding dimensions.
var_in = {"x": [n,1]}


def comb_fn(X_struct):
    x = X_struct.x
    x.requires_grad_(True)
    
    # objective function
    f = x.T@A@x

    # inequality constraint, matrix form
    ci = None
    
    # equality constraint 
    ce = GeneralStruct()
    ce.c1 = x.T@x-1

    return [f,ci,ce]


## User Options
Specify user-defined options for NCVX

In [4]:
opts = Options()
opts.print_frequency = 10
opts.x0 = torch.randn((n,1)).to(device=device, dtype=torch.double)
opts.mu0 = 0.1 # increase penalty contribution
opts.opt_tol = 1e-6

## Main Algorithm

In [5]:
start = time.time()
soln = ncvx(combinedFunction = comb_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════╗
║  NCVX requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.              ║
║  To disable this notice, set opts.quadprog_info_msg = False                               ║
╚═══════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
NCVX: A User-Friendly and Scalable Package for Nonconvex Optimization in Machine Learning                        ║ 
Version 1.1.1                                                                                                    ║ 
MIT License Copyright (c) 2021 SUN Group @ UMN                                                                   ║ 
═════════════════════════════════════════════════════════════════